# Anonimização por Perturbação

## Importações e definições de variáveis/funções globais

In [225]:
import numpy as np
import pandas as pd
import itertools as it
from datetime import datetime as dt
from math import radians, cos, sin, asin, sqrt
import random
from IPython.display import clear_output
import scipy.stats as stats

filename        = "Dataset_Covid_CE.csv"
filename_out    = "Dataset_Covid_CE_Anon_Per.csv"

df_municipios = pd.read_csv('ce-cities-lat-lon-format.csv')

## Leitura e limpeza do dataset inicial
Fazemos apenas uma limpeza básica para não modificar muito os resultados do algoritmo.

In [226]:
# definição dos semi-identificadores
semi_ids = [
    "municipioCaso",
    "dataNascimento",
]

# definição dos atributos sensíveis
sensitive = [
    "comorbidadeCardiovascularSivep",
    "comorbidadeDiabetesSivep",
]

# definição dos tipos
dtype = {
    "municipioCaso"                  : "str",
    "dataNascimento"                 : "str",
    "comorbidadeCardiovascularSivep" : "str",
    "comorbidadeDiabetesSivep"       : "str",
}

# definição das colunas de data
dates = [
    "dataNascimento",
]
date_parser = lambda x: pd.to_datetime(x, format="%Y-%m-%d", errors = 'coerce')

# pegar apenas os atributos desejados do dataset
# df = pd.read_csv(filename, usecols=semi_ids + sensitive, dtype=dtype)
df = pd.read_csv(filename, usecols=semi_ids + sensitive, dtype=dtype, parse_dates=dates, date_parser=date_parser)
# df = pd.read_csv(filename, nrows=500000, usecols=semi_ids + sensitive, dtype=dtype)
df = df.dropna(how="any", subset=semi_ids) # remover registros com semi_ids nulo
df['municipioCaso'] = df['municipioCaso'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') # remove acentuação
df


,municipioCaso,dataNascimento,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep
19,SOBRAL,2003-08-14,NaN,NaN
20,PACAJUS,1983-11-07,NaN,NaN
21,HORIZONTE,1982-01-14,NaN,NaN
25,FORTALEZA,1992-03-12,NaN,NaN
28,CAUCAIA,1970-03-06,NaN,NaN
...,...,...,...,...
1266272,FORTALEZA,1982-12-08,NaN,NaN
1266273,FORTALEZA,1980-12-02,NaN,NaN
1266274,FORTALEZA,1988-05-17,NaN,NaN
1266275,CRATEUS,1970-09-28,NaN,NaN


## Construção do dict de municipios e função haversine para calculo de distancias geográficas

In [227]:
municipios = df['municipioCaso'].unique().tolist()
municipios_dict = {x['municipioCaso']: (x['lat'], x['lon']) for i, x in df_municipios[['municipioCaso', 'lat', 'lon']].iterrows()}

def haversine(geo1, geo2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lat1, lon1 = geo1
    lat2, lon2 = geo2

    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

df_m_weights = df['municipioCaso'].value_counts().to_frame().rename(columns={'municipioCaso':'weight'})
df_m_weights['weight'] = stats.zscore(df_m_weights['weight']).flatten()
df_m_weights = df_m_weights.to_dict()['weight']

## Definição do algoritmo


In [228]:
def perturbate(df):
    """
    algoritmo de perturbação.

    df       -- dataframe para ser perturbado
    return   -- dataframe perturbado
    """

    cdf = df.copy()

    def per_date(value):
        """
        pega valores de ano-mês-dia (ex: 1986-04-26)
        pegar valor do ano (ex: 1986)
        dividir pelo dia + mês e arredonda (ex: 1986/26 = 77)
        adiciona valor ao original se mês é par, diminui se impar
        clamp para não dar datas maiores que hoje
        """
        year = value.year
        month = value.month
        day = value.day

        joker = round(year / (day + month))
        d1 = int(str(joker)[:1])

        new_date = value + pd.Timedelta(days=joker if month % 2 == 1 else -joker)

        return new_date if new_date <= dt.today() else dt.today().date()

    cdf['dataNascimento'] = cdf['dataNascimento'].map(per_date)

    def per_city1(value):
        """
        pega 10 cidades aleatóriamente das presentes no dataframe
        escolhe a cidade mais próxima da original dentre as selecionadas
        """

        choices = random.choices(municipios, k=10)

        orig_geo = municipios_dict[value]
        best_distance = 9999999
        best_choice = ""

        for choice in choices:
            choice_geo = municipios_dict[choice]
            distance = haversine(orig_geo, choice_geo)
            if distance < best_distance:
                best_choice = choice
                best_distance = distance
        return best_choice
    
    def per_city2(value):
        """
        pega 10 cidades aleatóriamente das presentes no dataframe
        compara os pesos das escolhas e pega a com peso maior
        """

        choices = random.choices(municipios, k=10)

        best_choice = ""
        best_weight = -999

        for choice in choices:
            choice_weight = df_m_weights[choice]
            if (choice_weight > best_weight):
                best_weight = choice_weight
                best_choice = choice
        return best_choice

    cdf['municipioCaso'] = cdf['municipioCaso'].map(per_city1)

    return cdf


## Aplicação do algoritmo

In [229]:
df

,municipioCaso,dataNascimento,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep
19,SOBRAL,2003-08-14,NaN,NaN
20,PACAJUS,1983-11-07,NaN,NaN
21,HORIZONTE,1982-01-14,NaN,NaN
25,FORTALEZA,1992-03-12,NaN,NaN
28,CAUCAIA,1970-03-06,NaN,NaN
...,...,...,...,...
1266272,FORTALEZA,1982-12-08,NaN,NaN
1266273,FORTALEZA,1980-12-02,NaN,NaN
1266274,FORTALEZA,1988-05-17,NaN,NaN
1266275,CRATEUS,1970-09-28,NaN,NaN


In [230]:
df_anon = perturbate(df)
df_anon

,municipioCaso,dataNascimento,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep
19,SOBRAL,2003-05-15,NaN,NaN
20,ITAITINGA,1984-02-25,NaN,NaN
21,MARANGUAPE,1982-05-26,NaN,NaN
25,PACATUBA,1992-07-23,NaN,NaN
28,EUSEBIO,1970-10-11,NaN,NaN
...,...,...,...,...
1266272,AQUIRAZ,1982-08-31,NaN,NaN
1266273,GUAIUBA,1980-07-14,NaN,NaN
1266274,GUAIUBA,1988-08-15,NaN,NaN
1266275,PORANGA,1970-11-20,NaN,NaN


In [231]:
print(df['dataNascimento'].describe(datetime_is_numeric=True))
print("")
print(df_anon['dataNascimento'].describe(datetime_is_numeric=True))

count                           911711
mean     1980-10-13 21:32:24.793909440
min                1691-02-27 00:00:00
25%                1968-11-09 00:00:00
50%                1982-10-19 00:00:00
75%                1994-03-07 00:00:00
max                2021-01-22 00:00:00
Name: dataNascimento, dtype: object

count                           911711
mean     1980-10-21 05:21:50.076987136
min                1690-12-31 00:00:00
25%                1968-11-17 00:00:00
50%                1982-10-27 00:00:00
75%                1994-03-02 00:00:00
max                2021-08-04 00:00:00
Name: dataNascimento, dtype: object


In [232]:
print(df['municipioCaso'].describe())
print("")
print(df_anon['municipioCaso'].describe())

count        911711
unique          184
top       FORTALEZA
freq         238318
Name: municipioCaso, dtype: object

count        911711
unique          184
top       FORTALEZA
freq          17190
Name: municipioCaso, dtype: object


## Gerar CSVs

In [233]:
df.to_csv(filename[:-4] + '_Clean.csv', index=False)
df_anon.to_csv(filename_out, index=False)